In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
drugs_file = "\\".join(os.path.dirname(os.path.abspath("__file__")).split("\\")[0:-1]) + "\\data\\Structured data\\drugs.csv"
protein_file = "\\".join(os.path.dirname(os.path.abspath("__file__")).split("\\")[0:-1]) + "\\data\\Structured data\\protien_id.csv"
drugs_side_effects_file = "\\".join(os.path.dirname(os.path.abspath("__file__")).split("\\")[0:-1]) + "\\data\\Structured data\\drugs-side-effects.xlsx"

In [3]:
# Read the CSV file into a DataFrame
df = pd.read_csv(protein_file)

# Split the 'Drug IDs' column into a list of IDs
df['Drug IDs'] = df['Drug IDs'].str.split('; ')

# Explode the DataFrame
df_exploded = df.explode('Drug IDs')

# Save the modified DataFrame to a new CSV file
new_file_path = "\\".join(os.path.dirname(os.path.abspath("__file__")).split("\\")[0:-1]) + "\\data\\Structured data\\target.parquet"  # Replace with your desired new file path
df_exploded.to_parquet(new_file_path, engine="fastparquet", index=False)

print("File saved as:", new_file_path)

File saved as: c:\Users\zeyad\Documents\Clinical-Trials-Knowledge-Graph-Project\data\Structured data\target.parquet


In [4]:
# Load the data from the CSV file
drugs_1_df = pd.read_csv(drugs_file)

# Load the data from the Excel file
drugs_2_df = pd.read_excel(drugs_side_effects_file)[['Medicine','Treated Diseases/Conditions','Side effects']]
target_dataset = pd.read_parquet("\\".join(os.path.dirname(os.path.abspath("__file__")).split("\\")[0:-1]) + "\\data\\Structured data\\target.parquet", engine='fastparquet')[['Name', 'Gene Name', 'GenBank Protein ID', 'GenBank Gene ID', 'UniProt ID', 'Uniprot Title', 'GenAtlas ID' ,'Species', 'Drug IDs']]



# Merge the dataframes

merged_df = drugs_1_df.merge(drugs_2_df, left_on='name', right_on='Medicine', how='left')
merged_df = merged_df.merge(target_dataset, left_on='ID', right_on='Drug IDs', how='left')


# Drop the redundant 'Medicine' column
merged_df.drop(['Medicine','Drug IDs'], axis=1, inplace=True)

# Identify rows with empty or 'Null' values in 'Treated Diseases/Conditions' and 'Side effects'
null_values_df = merged_df[((merged_df['Side effects'] == 'Null') & (merged_df['Treated Diseases/Conditions'] == 'Null')) |
                           (merged_df['Side effects'].isna() & merged_df['Treated Diseases/Conditions'].isna()) |
                            merged_df['Name'].isna() & 
                            merged_df['Gene Name'].isna() & 
                            merged_df['GenBank Protein ID'].isna() & 
                            merged_df['GenBank Gene ID'].isna() & 
                            merged_df['UniProt ID'].isna() & 
                            merged_df['Uniprot Title'].isna()  & 
                            merged_df['GenAtlas ID'].isna()  & 
                            merged_df['Species'].isna()]  
non_null_values_df = merged_df.drop(null_values_df.index).sort_values(by='name')

# Sort the null_values_df by 'name'
null_values_df = null_values_df.sort_values(by='name')

# Concatenate the dataframes back together
final_df = pd.concat([non_null_values_df, null_values_df], ignore_index=True)

final_df = final_df.drop_duplicates(keep='first')
final_df = final_df.replace("Null", np.nan)
final_df = final_df.replace("None", np.nan)
final_df = final_df.replace("null", np.nan)
final_df = final_df.replace("none", np.nan)
final_df = final_df[final_df['name'].str.lower() != np.nan]

final_df.reset_index(inplace=True, drop=True)


# Save the final dataframe to a new CSV file
final_df.to_parquet("\\".join(os.path.dirname(os.path.abspath("__file__")).split("\\")[0:-1]) + "\\data\\Structured data\\final_merged_drugs.parquet", engine="fastparquet", index=False)